# Device patch base
A Kubeflow pipeline running Ansible playbook from a git repo on a subset of devices

This Notebook is meant to run in a Notebook Server on the Teknoir platform

## Define the pipeline

In [8]:
import kfp
import kfp.dsl as dsl
import uuid
import os
import json
from kubernetes.client.models import V1EnvVar

project_id = os.getenv('PROJECT_ID', 'teknoir')
domain = os.getenv('DOMAIN', 'teknoir.cloud')
namespace = os.getenv('NAMESPACE', 'teknoir-ai')

pipeline_name='Patch device'
pipeline_description='A Kubeflow pipeline running Ansible playbook from a git repo on a subset of devices'
    
@dsl.pipeline(name=pipeline_name, description=pipeline_description)
def ansible_pipeline(playbook_repo, playbook_path, ansible_limit):
    """A pipeline to run arbitrary playbook from a public git repo."""

    volume_name = f"device-patch-{str(uuid.uuid4())[:6]}"
    mount_folder = "/workspace"

    # A working directory between steps
    wkdirop = dsl.VolumeOp(
        name=f'Create build workspace',
        resource_name=volume_name,
        size="500Mi",
        modes=dsl.VOLUME_MODE_RWO
    ).set_caching_options(False)

    playbook = dsl.ContainerOp(
        name="Clone Workspace",
        image="gcr.io/teknoir/edgebuild-kfp",
        command=["bash", "-c"],
        arguments=[f"""
set -eo pipefail
git clone https://github.com/teknoir/device-patch-base.git {mount_folder}
cd {mount_folder}
ls
./run_playbook.sh --repo {playbook_repo} --playbook {playbook_path} --limit {ansible_limit}
"""],
        pvolumes={mount_folder: wkdirop.volume}
    )

    playbook.execution_options.caching_strategy.max_cache_staleness = "P0D"

    


## Upload pipeline

In [9]:
pipeline_version_file = pipeline_file = 'device_patch.yaml'
workflow = kfp.compiler.Compiler().compile(pipeline_func=ansible_pipeline, package_path=pipeline_file)

client = kfp.Client(namespace='teknoir')
filter = json.dumps({'predicates': [{'key': 'name', 'op': 1, 'string_value': pipeline_name}]})
pipelines = client.pipelines.list_pipelines(filter=filter)

if not pipelines.pipelines:
    pipeline = client.pipeline_uploads.upload_pipeline(pipeline_file, name=pipeline_name, description=pipeline_description)
else:
    pipeline_version_name = pipeline_name + f' - {str(uuid.uuid4())[:6]}'
    pipeline_version = client.pipeline_uploads.upload_pipeline_version(pipeline_version_file,
                                                                       name=pipeline_version_name,
                                                                       pipelineid=pipelines.pipelines[0].id)

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').
